## 2017 Near-shore fishing activity

In [1]:
import pandas as pd
import geopandas
import os
os.getcwd()

'/home/jovyan'

In [2]:
df = pd.read_csv('combined2017.csv')

In [3]:
print(df.head())

         date  cell_ll_lat  cell_ll_lon       mmsi   hours  fishing_hours
0  2017-01-01          0.0         -2.6  306097000  1.6175         0.0000
1  2017-01-01          0.0       -171.5  338539000  0.5444         0.0000
2  2017-01-01          0.0       -152.6  359002000  1.3522         0.0000
3  2017-01-01          0.0        170.6  367170000  0.3972         0.0000
4  2017-01-01          0.0        170.7  367170000  0.6944         0.3891


In [4]:
len(df)

43665527

In [5]:
GFW2017Points = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.cell_ll_lon, df.cell_ll_lat), crs="EPSG:4326"
)

In [6]:
print(GFW2017Points.head())

         date  cell_ll_lat  cell_ll_lon       mmsi   hours  fishing_hours  \
0  2017-01-01          0.0         -2.6  306097000  1.6175         0.0000   
1  2017-01-01          0.0       -171.5  338539000  0.5444         0.0000   
2  2017-01-01          0.0       -152.6  359002000  1.3522         0.0000   
3  2017-01-01          0.0        170.6  367170000  0.3972         0.0000   
4  2017-01-01          0.0        170.7  367170000  0.6944         0.3891   

                     geometry  
0    POINT (-2.60000 0.00000)  
1  POINT (-171.50000 0.00000)  
2  POINT (-152.60000 0.00000)  
3   POINT (170.60000 0.00000)  
4   POINT (170.70000 0.00000)  


In [7]:
len(GFW2017Points)

43665527

### Now Clip to the 12nm territorial waters

In [8]:
EEZ12 = geopandas.read_file("eez_12nm_v3.shp")

In [9]:
len(EEZ12)

234

In [10]:
EEZ_12_2017 = GFW2017Points.clip(EEZ12)

In [11]:
print(EEZ_12_2017)

                date  cell_ll_lat  cell_ll_lon       mmsi   hours  \
9169839   2017-04-02        -46.6        170.0  512228000  0.2908   
36568639  2017-11-05        -46.6        170.0  512430000  0.1000   
24401202  2017-08-20        -46.6        170.0  512002236  0.4963   
4515340   2017-02-18        -46.6        170.0  512426000  1.5972   
2685095   2017-01-30        -46.6        170.0  512411000  0.0608   
...              ...          ...          ...        ...     ...   
27736170  2017-09-11         80.5         33.3  273370020  0.1297   
30905678  2017-09-29         80.5         33.3  257139000  0.0733   
28601592  2017-09-16         80.5         33.2  273550600  0.1722   
27736171  2017-09-11         80.5         33.2  273370020  0.3674   
30905665  2017-09-29         80.5         33.2  257139000  0.2633   

          fishing_hours                     geometry  
9169839          0.0000  POINT (170.00000 -46.60000)  
36568639         0.0000  POINT (170.00000 -46.60000)  
244012

In [12]:
len(EEZ_12_2017)

10082078

In [13]:
del GFW2017Points

In [14]:
del df

### Add flag and geartype

In [15]:
flag_gear_df = pd.read_csv('fishing-vessels-flagonly.csv')

In [16]:
flag_gear_df.head(2)

,mmsi,flag_gfw,vessel_class_gfw,fishing_hours_2019
0,40120000,GRC,trawlers,NaN
1,444002410,KOR,tuna_purse_seines,NaN


In [17]:
EEZ_12_2017 = pd.merge(EEZ_12_2017, flag_gear_df[['mmsi','flag_gfw','vessel_class_gfw']], on='mmsi', how='left')

In [18]:
EEZ_12_2017.head(2)

,date,cell_ll_lat,cell_ll_lon,mmsi,hours,fishing_hours,geometry,flag_gfw,vessel_class_gfw
0,2017-04-02,-46.6,170.0,512228000,0.2908,0.0,POINT (170.00000 -46.60000),NZL,trawlers
1,2017-11-05,-46.6,170.0,512430000,0.1000,0.0,POINT (170.00000 -46.60000),NZL,trawlers


In [19]:
len(EEZ_12_2017)

10082078

### Select only for trawler vessels

In [20]:
Trawl_12nm2017 = EEZ_12_2017[(EEZ_12_2017['vessel_class_gfw'] == 'trawlers')]

In [21]:
Trawl_12nm2017.head(2)

,date,cell_ll_lat,cell_ll_lon,mmsi,hours,fishing_hours,geometry,flag_gfw,vessel_class_gfw
0,2017-04-02,-46.6,170.0,512228000,0.2908,0.0,POINT (170.00000 -46.60000),NZL,trawlers
1,2017-11-05,-46.6,170.0,512430000,0.1000,0.0,POINT (170.00000 -46.60000),NZL,trawlers


In [22]:
len(Trawl_12nm2017)

5500589

### Do a spatial join to get the Trawler Points to have a territory name attached to it

In [23]:
EEZ12.head(2)

,MRGID,GEONAME,POL_TYPE,MRGID_TER1,TERRITORY1,MRGID_SOV1,SOVEREIGN1,ISO_TER1,X_1,Y_1,MRGID_EEZ,AREA_KM2,ISO_SOV1,UN_SOV1,UN_TER1,geometry
0,49010.0,Belgian 12 NM,12NM,14.0,Belgium,14.0,Belgium,BEL,2.86467,51.32730,3293.0,1444.0,BEL,56,56.0,"POLYGON ((3.10612 51.53901, 3.11336 51.53531, ..."
1,49012.0,Albanian 12 NM,12NM,2153.0,Albania,2153.0,Albania,ALB,19.34000,40.82508,5670.0,5455.0,ALB,8,8.0,"POLYGON ((20.00835 39.69182, 20.00810 39.69187..."


In [25]:
Trawl_w_territory = Trawl_12nm2017.sjoin(EEZ12, how ='left', predicate ='within')

In [26]:
Trawl_w_territory.head(2)

,date,cell_ll_lat,cell_ll_lon,mmsi,hours,fishing_hours,geometry,flag_gfw,vessel_class_gfw,index_right,...,MRGID_SOV1,SOVEREIGN1,ISO_TER1,X_1,Y_1,MRGID_EEZ,AREA_KM2,ISO_SOV1,UN_SOV1,UN_TER1
0,2017-04-02,-46.6,170.0,512228000,0.2908,0.0,POINT (170.00000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0
1,2017-11-05,-46.6,170.0,512430000,0.1000,0.0,POINT (170.00000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0


In [27]:
Trawl_w_territory[["date","mmsi","hours","fishing_hours","geometry","flag_gfw","vessel_class_gfw","ISO_TER1"]].head()

,date,mmsi,hours,fishing_hours,geometry,flag_gfw,vessel_class_gfw,ISO_TER1
0,2017-04-02,512228000,0.2908,0.0,POINT (170.00000 -46.60000),NZL,trawlers,NZL
1,2017-11-05,512430000,0.1000,0.0,POINT (170.00000 -46.60000),NZL,trawlers,NZL
2,2017-08-20,512002236,0.4963,0.0,POINT (170.00000 -46.60000),NZL,trawlers,NZL
3,2017-02-18,512426000,1.5972,0.0,POINT (170.00000 -46.60000),NZL,trawlers,NZL
4,2017-01-30,512411000,0.0608,0.0,POINT (170.00000 -46.60000),NZL,trawlers,NZL


In [28]:
len(Trawl_w_territory)

5500589

### Group by the EEZ polygon and sum fishing hours and hours present

In [30]:
SumFishing_Hours12nm =Trawl_w_territory.groupby(['ISO_TER1'])['hours','fishing_hours'].sum().reset_index()

/tmp/ipykernel_679/1965765034.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  SumFishing_Hours12nm =Trawl_w_territory.groupby(['ISO_TER1'])['hours','fishing_hours'].sum().reset_index()


In [39]:
len(SumFishing_Hours12nm)

133

In [38]:
SumFishing_Hours12nm.head(5)

,ISO_TER1,hours,fishing_hours
0,ABW,644.8333,90.8621
1,AGO,68177.8657,9232.2787
2,AIA,0.2041,0.0000
3,ALB,3108.8390,674.8196
4,ARE,208.5620,46.5800


In [40]:
#Turn into a dataframe 
SumFishing_Hours12nm = pd.DataFrame(SumFishing_Hours12nm)

In [ ]:
SumFishing_Hours12nm.to_csv('SumFishing_Hours12nm.csv')

### Get rid of 0 hours present points and group by and then count # of vessels in each EEZ polygon

In [42]:
Trawl_w_territoryTable = pd.DataFrame(Trawl_w_territory)

In [43]:
import numpy as np

In [44]:
Trawl_w_territoryTable.replace(0, np.nan, inplace=True)

In [45]:
HoursTrawlnoNAN = Trawl_w_territoryTable.dropna(subset=['hours'])

In [46]:
HoursTrawlnoNAN.head()

,date,cell_ll_lat,cell_ll_lon,mmsi,hours,fishing_hours,geometry,flag_gfw,vessel_class_gfw,index_right,...,MRGID_SOV1,SOVEREIGN1,ISO_TER1,X_1,Y_1,MRGID_EEZ,AREA_KM2,ISO_SOV1,UN_SOV1,UN_TER1
0,2017-04-02,-46.6,170.0,512228000,0.2908,NaN,POINT (170.00000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0
1,2017-11-05,-46.6,170.0,512430000,0.1000,NaN,POINT (170.00000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0
2,2017-08-20,-46.6,170.0,512002236,0.4963,NaN,POINT (170.00000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0
3,2017-02-18,-46.6,170.0,512426000,1.5972,NaN,POINT (170.00000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0
4,2017-01-30,-46.6,170.0,512411000,0.0608,NaN,POINT (170.00000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0


In [47]:
VesselsPresentCount = HoursTrawlnoNAN.groupby(['ISO_TER1'])['hours'].count()

In [50]:
VesselsPresentCount.head(10)

ISO_TER1
ABW      123
AGO     8297
AIA        1
ALB      581
ARE      115
ARG    53444
ATF      297
AUS    10638
BEL    29176
BEN        2
Name: hours, dtype: int64

In [49]:
len(VesselsPresentCount)

133

### Get rid of points with 0 fishing hours from OG Trawl file and then group by and count # of vessels in each EEZ polygon

In [51]:
FishingTrawlnoNAN = Trawl_w_territoryTable.dropna(subset=['fishing_hours'])

In [54]:
FishingTrawlnoNAN.head(2)

,date,cell_ll_lat,cell_ll_lon,mmsi,hours,fishing_hours,geometry,flag_gfw,vessel_class_gfw,index_right,...,MRGID_SOV1,SOVEREIGN1,ISO_TER1,X_1,Y_1,MRGID_EEZ,AREA_KM2,ISO_SOV1,UN_SOV1,UN_TER1
51,2017-05-29,-46.6,169.7,512002843,2.5919,2.5919,POINT (169.70000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0
72,2017-05-24,-46.6,169.6,512002843,0.2494,0.2494,POINT (169.60000 -46.60000),NZL,trawlers,213.0,...,2227.0,New Zealand,NZL,136.49119,-41.49446,8455.0,180085.0,NZL,554.0,554.0


In [55]:
VesselsFishingCount = FishingTrawlnoNAN.groupby(['ISO_TER1'])['fishing_hours'].count()

In [56]:
VesselsFishingCount.head(10)

ISO_TER1
ABW       51
AGO     2336
ALB      217
ARE       18
ARG     9965
ATF       98
AUS     4508
BEL    15356
BEN        1
BGR     2662
Name: fishing_hours, dtype: int64

In [57]:
len(VesselsFishingCount)

108

### Export remaining two dataframes as csvs

In [58]:
VesselsFishingCount.to_csv('VesselsFishingCount.csv')

In [59]:
VesselsPresentCount.to_csv('VesselsPresentCount.csv')